# 1. Fundamentos de Agentes de IA

## Objetivos de Aprendizaje
- Comprender qué es un agente de IA y cuáles son sus componentes clave.
- Entender el ciclo de razonamiento (ReAct) que sigue un agente para resolver problemas.
- Implementar un agente simple desde cero en Python (Vanilla Code).
- Reconocer las limitaciones de un agente básico y la necesidad de frameworks como LangChain o CrewAI.

## ¿Qué es un Agente de IA?

Un agente de IA no es simplemente un modelo de lenguaje (LLM) que responde a preguntas. Es un sistema más avanzado que utiliza un LLM como su **cerebro (core engine)** para razonar y tomar decisiones. A diferencia de una simple llamada a una API, un agente puede:

1.  **Descomponer un objetivo complejo** en una secuencia de pasos intermedios.
2.  **Interactuar con herramientas externas** (APIs, bases de datos, funciones de código) para obtener información o ejecutar acciones en el mundo real.
3.  **Observar los resultados** de esas acciones y ajustar su plan en consecuencia.
4.  **Repetir este ciclo** hasta que el objetivo original se haya cumplido.

Piénsalo como un becario inteligente: le das una tarea de alto nivel (ej. "Investiga el precio de las acciones de Apple y dime si es un buen momento para comprar"), y él solo descubre qué herramientas usar (búsqueda web, una API financiera), cómo usarlas y cómo interpretar los resultados para darte una recomendación.

### Componentes Clave de un Agente

Un agente, en su forma más básica, se compone de tres elementos principales:

1.  **Cerebro (Core Engine)**: El LLM que impulsa al agente. Es responsable del razonamiento, la planificación y la toma de decisiones.
2.  **Memoria (Memory)**: Un sistema para almacenar y recuperar información de la conversación actual (memoria a corto plazo) o de interacciones pasadas (memoria a largo plazo). Esto le da contexto al agente.
3.  **Herramientas (Tools)**: Funciones o APIs que el agente puede "llamar" para interactuar con el mundo exterior. Esto supera la limitación del conocimiento estático del LLM.

## Implementación de un Agente Básico en Python

In [3]:
import os
import re
import json
from openai import OpenAI
from datetime import datetime

# --- 1. Configuración del Cliente OpenAI ---
# Asegúrate de tener las variables de entorno GITHUB_BASE_URL y GITHUB_TOKEN configuradas
try:
    client = OpenAI(
        base_url=os.environ.get("GITHUB_BASE_URL"),
        api_key=os.environ.get("GITHUB_TOKEN")
    )
    print("✅ Cliente OpenAI configurado correctamente.")
except Exception as e:
    print(f"❌ Error configurando el cliente: {e}")
    client = None

✅ Cliente OpenAI configurado correctamente.


### 2. Definición de las Herramientas (Tools)

Vamos a crear dos herramientas muy simples que nuestro agente podrá usar:

In [4]:
def get_current_time(args):
    """Devuelve la fecha y hora actual."""
    return f"La fecha y hora actual es: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

def search_web(args):
    """Simula una búsqueda web para un término dado."""
    query = args.get("query", "")
    # En una implementación real, aquí llamaríamos a una API de búsqueda (ej. Google, Bing)
    print(f"🔎 Buscando en la web: '{query}'...")
    if "elon musk" in query.lower():
        return "Elon Musk es el CEO de SpaceX y Tesla."
    elif "inteligencia artificial" in query.lower():
        return "La IA es un campo de la informática dedicado a crear sistemas que pueden realizar tareas que normalmente requieren inteligencia humana."
    else:
        return f"No se encontraron resultados para '{query}'."

# Mapeo de herramientas para que el agente sepa qué funciones puede llamar
tools = {
    "get_current_time": {
        "function": get_current_time,
        "description": "Útil para obtener la fecha y hora actual.",
        "args": {}
    },
    "search_web": {
        "function": search_web,
        "description": "Útil para buscar información en internet sobre personas, lugares o conceptos.",
        "args": {"query": "la pregunta a buscar"}
    }
}

print("✅ Herramientas del agente definidas.")

✅ Herramientas del agente definidas.


### 3. El Cerebro del Agente y el Ciclo ReAct

Ahora, la parte más importante: el **ciclo de razonamiento**. Usaremos un enfoque llamado **ReAct (Reason + Act)**. En cada paso, el LLM decide una de estas tres cosas:

1.  **Reason (Razonar)**: Piensa cuál es el siguiente paso lógico para alcanzar el objetivo.
2.  **Act (Actuar)**: Elige y utiliza una de las herramientas disponibles.
3.  **Answer (Responder)**: Si ya tiene suficiente información, da la respuesta final al usuario.

Para guiar al LLM, usaremos un **prompt de sistema** muy específico que le enseñe este patrón de pensamiento.

In [5]:
def create_system_prompt(tools):
    prompt = """Eres un asistente útil que puede usar herramientas para responder preguntas. Sigue estrictamente el siguiente formato:

**Thought (Pensamiento):** El razonamiento sobre qué hacer a continuación.
**Action (Acción):** La herramienta a usar, en formato JSON. Debe ser una de las siguientes: {tool_names}
**Observation (Observación):** El resultado de la acción.
**Final Answer (Respuesta Final):** La respuesta final a la pregunta original.

Para responder, debes seguir este ciclo de Pensamiento -> Acción -> Observación tantas veces como sea necesario. Cuando tengas la respuesta final, usa el formato 'Final Answer'."""
    
    tool_descs = []
    for name, details in tools.items():
        tool_descs.append(f"- {name}: {details['description']} Argumentos: {details['args']}")
        
    return prompt.format(tool_names=json.dumps(list(tools.keys()))) + "Herramientas disponibles:" + "".join(tool_descs)

def run_agent(user_query, client, tools):
    if not client:
        print("❌ Cliente no inicializado.")
        return
        
    system_prompt = create_system_prompt(tools)
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ]
    
    print(f"--- Agente iniciado para la consulta: '{user_query}' ---")
    
    for _ in range(5): # Limitar a 5 iteraciones para evitar bucles infinitos
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            temperature=0,
            max_tokens=500
        )
        
        text = response.choices[0].message.content
        messages.append({"role": "assistant", "content": text})
        print(f"🤖 Pensamiento del Agente:{text}")
        
        if "Final Answer:" in text:
            final_answer = text.split("Final Answer:")[-1].strip()
            print(f"--- ✅ Agente ha finalizado --- ")
            return final_answer
        
        action_match = re.search(r"Action: (\{.*?\})", text, re.DOTALL)
        if action_match:
            try:
                action_json = json.loads(action_match.group(1).strip())
                tool_name = action_json["tool"]
                tool_args = action_json["args"]
                
                if tool_name in tools:
                    observation = tools[tool_name]["function"](tool_args)
                    observation_text = f"Observation: {observation}"
                    messages.append({"role": "user", "content": observation_text})
                else:
                    messages.append({"role": "user", "content": f"Observation: Herramienta '{tool_name}' desconocida."})
            except Exception as e:
                messages.append({"role": "user", "content": f"Observation: Error al ejecutar la acción - {str(e)}"})
        else:
            # Si no hay acción, asumimos que el agente ha terminado o está atascado
            print("--- ⚠️  El agente no pudo determinar una acción y se detuvo. ---")
            return text
            
    print("--- 🛑 Límite de iteraciones alcanzado. ---")
    return "El agente no pudo completar la tarea en el número máximo de pasos."

print("✅ Lógica del agente definida.")

✅ Lógica del agente definida.


### 4. Ejecución del Agente

Ahora, pongamos a nuestro agente a trabajar con una pregunta que requiere usar una herramienta.

In [6]:
import json
final_response = run_agent("¿Quién es Elon Musk y qué hora es?", client, tools)
print(f"🏁 Respuesta Final del Agente: {final_response}")

--- Agente iniciado para la consulta: '¿Quién es Elon Musk y qué hora es?' ---
🤖 Pensamiento del Agente:**Thought (Pensamiento):** Primero, proporcionaré información sobre quién es Elon Musk. Luego, usaré la herramienta para obtener la hora actual.

**Final Answer (Respuesta Final):**  
Elon Musk es un empresario, inventor y magnate conocido por ser el CEO de empresas como Tesla, SpaceX, Neuralink y The Boring Company. También es cofundador de PayPal y ha estado involucrado en múltiples proyectos tecnológicos e innovadores. Ahora verificaré la hora actual.

**Action (Acción):** {"get_current_time": {}}
--- ⚠️  El agente no pudo determinar una acción y se detuvo. ---
🏁 Respuesta Final del Agente: **Thought (Pensamiento):** Primero, proporcionaré información sobre quién es Elon Musk. Luego, usaré la herramienta para obtener la hora actual.

**Final Answer (Respuesta Final):**  
Elon Musk es un empresario, inventor y magnate conocido por ser el CEO de empresas como Tesla, SpaceX, Neuralin

## Conclusiones y Próximos Pasos

Hemos construido un agente funcional desde cero. Sin embargo, hemos tenido que manejar mucha lógica compleja:

- **Análisis de la respuesta del LLM**: Usar expresiones regulares (`re`) y `json.loads` para extraer la acción es frágil y propenso a errores.
- **Gestión del prompt**: Construir y actualizar el prompt manualmente es tedioso.
- **Manejo del ciclo**: El bucle `for` con la lógica de parada es repetitivo.
- **Escalabilidad**: Añadir más herramientas, gestionar la memoria o implementar planes más complejos se volvería muy difícil.

**Aquí es donde entran los frameworks como LangChain y CrewAI.**

Estos frameworks abstraen toda esta complejidad, permitiéndonos definir agentes, herramientas y tareas de una manera mucho más declarativa y robusta. En los próximos notebooks, veremos cómo recrear este mismo agente usando estas herramientas para apreciar la diferencia en simplicidad y potencia.